In [2]:
# install pyscf
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d

# basis
mol = gto.M(
    verbose = 0,
    atom = '''
    o    0    0.       0.
    h    0    -0.757   0.587
    h    0    0.757    0.587''',
    basis = '6-31g')

nd = 13
func = lambda x, y, z: numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))

# nd = 4
# func = lambda x, y, z: numpy.array([ numpy.exp(-(x**2 + y**2 + z**2) * 5), \
#                                      numpy.exp(-((x - 1/2)**2 + (y - 1/3)**2 + (z - 3/5)**2) * 10), \
#                                      numpy.exp(-((x + 1/2)**2 + (y + 1/3)**2 + (z + 3/5)**2) * 20), \
#                                      numpy.exp(-((x + 1/4)**2 + (y - 1/5)**2 + (z - 4/5)**2) * 2)]).reshape(nd,-1).transpose()

# initialize tree parameters
dom = numpy.array([[-5], [5], [-5], [5], [-5], [5]])
f = {
    'domain': dom, 
    'tol': 1.0e-4,
    'nSteps': 15,
    'level': numpy.array([0]),
    'height': numpy.array([0]),
    'id': numpy.array([0]), # 
    'parent': numpy.array([0]),
    'children': numpy.zeros((8,1)), 
    'coeffs': [],
    'col': numpy.array([0]),
    'row': numpy.array([0]),
    'n': 8,
    'checkpts': numpy.array([[0,    0,     0],
                             [0, -0.757, 0.757],
                             [0,  0.587, 0.587]]),
    'rint': numpy.array([[] for k in range(nd)]),
    'vmax': numpy.array([[] for k in range(nd)])                         
  }


f, rint = buildBreadthFirst3d(f, func)

# compute basis for plotting
numpts = 51 # this needs to be consistent with the resolution in plot3dtree.m
xx, yy, zz = numpy.meshgrid(numpy.linspace(dom[0,0],dom[1,0],numpts),numpy.linspace(dom[2,0],dom[3,0],numpts),numpy.linspace(dom[4,0],dom[5,0],numpts),indexing='ij')
v = func(xx.flatten(),yy.flatten(),zz.flatten())

savemat('pyscf_test.mat', {'numpts': numpts, 'v': v, 'xx': xx, 'yy': yy, 'zz': zz, 'rint': rint, 'fdomain': f['domain'], 'flevel': f['level'], 'fchildren': f['children'], 'fheight': f['height'], 'fid': f['id'], 'frint': f['rint'], 'ftol': f['tol'], 'fcheckpts': f['checkpts']})